This Notebook is to extract the input image's value to control points.<br>

Why doing that? Because we can directly use the control poits with image's value to train classifier,<br>
Otherwise a lot of time would be wasted during classifier training on "Extrating image value to points"<br>

Specifically, we:<br>
1) stack all input images into a multiband image;<br>
2) extract the multiband image value to control points;<br>
3) export the points of extraction

In [1]:
import ee
import datetime
import os
import itertools
import sys

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

##### Define_Basic_Parameters

In [3]:
# define the year range
year_start = [i for i in range(1990,2018,3)]
year_end   = [i for i in range(1992,2020,3)]

year_name = [f'{i[0]}_{i[1]}' for i in zip(year_start,year_end)]

In [4]:
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [5]:
year_name

['1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019']

##### Prepare needed images

In [6]:
# import Landsat img
Landsat_img = [ee.Image(f"users/wang8052664/Cloud_Free_Img/Landsat_cloud_free_{year}")\
                 .clip(North_China_Plain)  for year in year_name]

# import Fourier img   
Fourier_img = [ee.Image(f"users/Jinzhu_Deakin/North_China_Plain/Fourier_img_harmonic_3/Fourier_img_{year}_harmonic_3")\
                     .clip(North_China_Plain)  for year in year_name[:5]] + \
              [ee.Image(f"users/wang8052664/North_China_Plain/Fourier_imgs/Fourier_img_{year}_harmonic_3")\
                     .clip(North_China_Plain)  for year in year_name[5:]] 

# Import the Index-mean Img
NDVI_img = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_NDVI").rename('Mean_NDVI')  
            for year in year_name ]
NDBI_img = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_NDBI").rename('Mean_NDBI')  
            for year in year_name ]
EVI_img  = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_EVI").rename('Mean_EVI')  
            for year in year_name ]

# prepare the climate data
Climate_mean = [ee.Image("users/wangjinzhulala/North_China_Plain_Python/Climate_data/Mean_1990_1992")
               .clip(North_China_Plain)  for year in year_name]

# Import DEM/SLOPE Img
DEM   = [ee.Image("USGS/SRTMGL1_003").rename('DEM')] * len(Landsat_img)
SLOPE = [ee.Terrain.slope(DEM).rename('SLOPE')] * len(Landsat_img)

In [7]:
# stack all Imput_Img together
Stack_img = [ee.Image(img) for img in zip(Landsat_img,Fourier_img,NDVI_img,NDBI_img,EVI_img,Climate_mean,DEM,SLOPE)]

##### Img value extraction and export

In [8]:
# Get the training sample
Training_sample = ee.FeatureCollection('users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Control_points')

In [9]:
# define the gee-asset path for exporting
path = 'users/wangjinzhulala/North_China_Plain_Python/Sample_extract_img'


for year,img in list(zip(year_name,Stack_img)):
    
    name   = 'Control_sample_ext_img'
    sample = Training_sample
    
    Ext_sample = img.sampleRegions(collection  = sample, 
                                    properties = ['Built'], 
                                    scale      = 30, 
                                    geometries = True)
    
    # exporting
    task = ee.batch.Export.table.toAsset(   collection  = Ext_sample,
                                            description = f'{name}_{year}',
                                            assetId     = f'{path}/{name}_{year}')
    task.start()

    
    # print out the process
    print(f'{name}_{year}')

Control_sample_ext_img_1990_1992
Control_sample_ext_img_1993_1995
Control_sample_ext_img_1996_1998
Control_sample_ext_img_1999_2001
Control_sample_ext_img_2002_2004
Control_sample_ext_img_2005_2007
Control_sample_ext_img_2008_2010
Control_sample_ext_img_2011_2013
Control_sample_ext_img_2014_2016
Control_sample_ext_img_2017_2019
